In [1]:
# remove this
import sys
#sys.path.append("/Users/harrison37/Work/alpine/ascent/install-debug/python-modules/")
sys.path.append("/Users/larsen30/research/test_builds/release/ascent/install/python-modules")

In [2]:
# cleanup any old results
!./cleanup.sh

In [3]:
# Ascent imports
# ascent + conduit imports
import conduit
import conduit.blueprint
import ascent

import numpy as np

# Jupyter imports
from IPython.display import Image
# helper we use when displaying results in the notebook
img_display_width = 500

In [4]:
# Setup Actions to render our input datasets

# setup actions
actions = conduit.Node()
add_act = actions.append()
add_act["action"] = "add_scenes"
scenes = add_act["scenes"]
scenes["s1/plots/p1/type"] = "pseudocolor";
scenes["s1/plots/p1/field"] = "iters";
scenes["s1/image_width"] = 500
scenes["s1/image_height"] = 500

# print our full actions tree
print(actions.to_yaml())

# Sweep the juila set to create a sample time varying
# data set

nsteps = 20

# we will generate and rendering juila set examples linearly 
# interpolated between to known boxes

# starting box
extents_start = np.array( [-2.0, 2.0,  # x_min, x_max
                          -2.0, 2.0], # y_min, y_max
                           dtype=np.float64)

# ending box
extents_end = np.array( [-0.8, -0.4,  # x_min, x_max
                         0.0, 0.4],  # y_min, y_max
                         dtype=np.float64)

extents_delta = extents_end - extents_start

a = ascent.Ascent()
a.open()

info = conduit.Node()

for i in range(nsteps+1):
    extents = extents_start + extents_delta * (float(i) / float(nsteps))
    print("Generating example with extents {}".format(extents))
    # create new dataset
    mesh = conduit.Node()
    conduit.blueprint.mesh.examples.julia(250,   250, # nx, ny
                                         extents[0], extents[1], # x range
                                         extents[2], extents[3], # y range
                                         0.285, 0.01, # c value
                                         mesh)
    # publish mesh to ascent
    a.publish(mesh)
    # render with ascent
    scenes["s1/image_name"] = "out_julia_%04d" % i;
    a.execute(actions)
    a.info(info)
    print("Created {}".format(info["images"][0]["image_name"]))
# close ascent
a.close()


- 
  action: "add_scenes"
  scenes: 
    s1: 
      plots: 
        p1: 
          type: "pseudocolor"
          field: "iters"
      image_width: 500
      image_height: 500

Generating example with extents [-2.  2. -2.  2.]
Created out_julia_0000.png
Generating example with extents [-1.94  1.88 -1.9   1.92]
Created out_julia_0001.png
Generating example with extents [-1.88  1.76 -1.8   1.84]
Created out_julia_0002.png
Generating example with extents [-1.82  1.64 -1.7   1.76]
Created out_julia_0003.png
Generating example with extents [-1.76  1.52 -1.6   1.68]
Created out_julia_0004.png
Generating example with extents [-1.7  1.4 -1.5  1.6]
Created out_julia_0005.png
Generating example with extents [-1.64  1.28 -1.4   1.52]
Created out_julia_0006.png
Generating example with extents [-1.58  1.16 -1.3   1.44]
Created out_julia_0007.png
Generating example with extents [-1.52  1.04 -1.2   1.36]
Created out_julia_0008.png
Generating example with extents [-1.46  0.92 -1.1   1.28]
Created out_

KeyboardInterrupt: 

In [ ]:
Image("out_julia_0000.png",width=img_display_width)

In [ ]:
pattern = "out_julia_%04d.png"
data = []
for i in range(nsteps):
    with open(pattern % i, "rb") as f:
        data.append(f.read())

In [ ]:
import ipywidgets as widgets
 
image = widgets.Image(value=data[0],
    width=500,
    height=500,
    format="jpg")
 
play = widgets.Play(value=0,
    min=0,
    max=len(data)-1,
    step=1,
    interval=500)
 
def update(change):
    index = change.owner.value
    image.value = data[index]
 
play.observe(update)
 
widgets.VBox([image, play])

In [ ]:
from ascent_tutorial_jupyter_utils import ImageSeqViewer

In [ ]:
ifiles = [ "out_julia_%04d.png" % i for i in range(20) ]

ifiles


In [ ]:
print(ifiles)

In [ ]:
v = ImageSeqViewer(ifiles)

In [ ]:
dir(v)

In [ ]:
v.run()